<a href="https://colab.research.google.com/github/yujinc129-oss/Drama_data_project/blob/main/04_encoding_%26_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn. ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer


In [2]:
drama =pd.read_json('https://github.com/yujinc129-oss/Drama_data_project/raw/refs/heads/main/data/processed/03_feature', orient="records")

#인코딩

In [3]:
mlb = MultiLabelBinarizer()

fmt = mlb.fit_transform(drama['genres'])
gen = pd.DataFrame(fmt, columns=mlb.classes_, index=drama.index)


mlb = MultiLabelBinarizer()

fmt = mlb.fit_transform(drama['day'])
day = pd.DataFrame(fmt, columns=mlb.classes_, index=drama.index)

mlb = MultiLabelBinarizer()

fmt = mlb.fit_transform(drama['network'])
net = pd.DataFrame(fmt, columns=mlb.classes_, index=drama.index)


In [4]:
net

,ETC,JTBC,KBS,MBC,NETFLIX,OCN,SBS,TVN
0,0,1,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,1
3,0,0,0,1,1,0,0,0
4,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
2212,0,0,1,0,0,0,0,0
2213,0,0,0,1,0,0,0,0
2214,1,0,0,0,0,0,0,0
2215,0,0,0,1,0,0,0,0


In [5]:
others = pd.get_dummies(drama[['role','gender']])

In [6]:
X = pd.concat([drama['age'],gen, day, net, others], axis=1)


In [7]:
X

,age,comedy,drama,etc,romance,sf,society,thriller,friday,monday,...,KBS,MBC,NETFLIX,OCN,SBS,TVN,role_Main Role,role_Support Role,gender_Female,gender_Male
0,34,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,True,False,False,True
1,34,0,1,1,0,0,0,0,1,0,...,0,0,1,0,0,0,True,False,False,True
2,33,0,1,0,1,0,0,0,0,1,...,0,0,0,0,0,1,True,False,False,True
3,32,0,1,0,1,0,0,0,0,0,...,0,1,1,0,0,0,True,False,False,True
4,31,0,1,0,1,0,0,0,1,0,...,0,0,1,0,0,0,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2212,40,1,1,0,1,0,0,0,1,0,...,1,0,0,0,0,0,False,True,False,True
2213,32,0,1,0,1,0,0,0,1,0,...,0,1,0,0,0,0,True,False,False,True
2214,27,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,True,False,False,True
2215,32,0,1,0,1,0,1,1,1,0,...,0,1,0,0,0,0,True,False,False,True


In [8]:
X_data = X.copy()
data_processed = pd.concat([X_data, drama[['start airing','main_ratio','platform_tier']]], axis=1)

In [9]:
#X변수
X = data_processed
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2217 entries, 0 to 2216
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                2217 non-null   int64  
 1   comedy             2217 non-null   int64  
 2   drama              2217 non-null   int64  
 3   etc                2217 non-null   int64  
 4   romance            2217 non-null   int64  
 5   sf                 2217 non-null   int64  
 6   society            2217 non-null   int64  
 7   thriller           2217 non-null   int64  
 8   friday             2217 non-null   int64  
 9   monday             2217 non-null   int64  
 10  saturday           2217 non-null   int64  
 11  sunday             2217 non-null   int64  
 12  thursday           2217 non-null   int64  
 13  tuesday            2217 non-null   int64  
 14  wednesday          2217 non-null   int64  
 15  ETC                2217 non-null   int64  
 16  JTBC               2217 

In [10]:
#y 변수
y= drama['score']

#train/test split

In [11]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42)

#Linear Regression

In [12]:
linear = LinearRegression()
linear.fit(X_tr,y_tr)

LinearRegression()

In [13]:
Xpred = linear.predict(X_tr)


In [14]:
cv = cross_validate(linear, X_tr, y_tr, cv=5, n_jobs=1, return_train_score=True)

In [15]:
cv

{'fit_time': array([0.00503874, 0.00498056, 0.00510573, 0.00504041, 0.00490808]),
 'score_time': array([0.00287175, 0.00263023, 0.00251269, 0.00279403, 0.00261736]),
 'test_score': array([0.103098  , 0.17635233, 0.17950947, 0.18095582, 0.17473131]),
 'train_score': array([0.21652719, 0.1970414 , 0.19764744, 0.19431445, 0.19847372])}

In [ ]:
#결과
print(f'TRAIN_SCORE : {cv["train_score"].mean()}  |  TEST_SCORE : {cv["test_score"].mean()}')
print('MAE :',mean_absolute_error(y_tr, Xpred))
print('MSE :',mean_squared_error(y_tr, Xpred))
print('R2 :',r2_score(y_tr, Xpred))

TRAIN_SCORE : 0.20080084119045577  |  TEST_SCORE : 0.16292938541224747
MAE : 0.19035330515642893
MSE : 0.0646255953265736
R2 : 0.1971937561509629


#Randomforest

In [ ]:
randomf = RandomForestRegressor()

In [ ]:
randomf.fit(X_tr, y_tr)
X_pred = randomf.predict(X_tr)

In [ ]:
scoring_metrics = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']
cv = cross_validate(randomf, X_tr, y_tr, n_jobs=-1, return_train_score=True, scoring=scoring_metrics )


train_r2_mean = cv['train_r2'].mean()
train_mse_mean = -cv['train_neg_mean_squared_error'].mean() # 음수이므로 -1을 곱해 복원
train_mae_mean = -cv['train_neg_mean_absolute_error'].mean()

test_r2_mean = cv['test_r2'].mean()
test_mse_mean = -cv['test_neg_mean_squared_error'].mean() # 음수이므로 -1을 곱해 복원
test_mae_mean = -cv['test_neg_mean_absolute_error'].mean() # 음수이므로 -1을 곱해 복원
#결과
print(f'TRAIN MAE : {train_mae_mean} / TEST MAE : {test_mae_mean}' )
print(f'TRAIN MSE : {train_mse_mean} / TEST MSE : {test_mse_mean}')
print(f'TRAIN R2 : {train_r2_mean} / TEST R2 : {test_r2_mean}')

TRAIN MAE : 0.040377754272644416 / TEST MAE : 0.11046604025887945
TRAIN MSE : 0.003639104377695053 / TEST MSE : 0.025742196799872018
TRAIN R2 : 0.9547713534216417 / TEST R2 : 0.6809959969022731


##RandomizedSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import scipy

In [ ]:
# #params = {'max_depth' : scipy.stats.randint(100,150),
#           'min_samples_leaf' : scipy.stats.randint(2,10),
#           }

#randcv = RandomizedSearchCV(randomf, params, n_iter= 100, n_jobs=-1, return_train_score=True)

#randcv.fit(X_tr, y_tr)

#GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
16, 2, 400

(16, 2, 400)

In [ ]:
params = { 'n_estimators': [600],  # 이전 최적값으로 고정
    'max_depth': [10, 12, 15], # 이전 최적값(18)보다 의도적으로 낮게 설정
    'min_samples_leaf': [5, 10, 15]                 # 간격을 두고 3개 값 테스트
    #'min_samples_split' : [2, 5, 10],                # 기존 범위에서 일부만 선택
    #'max_features': ['sqrt']
           }

grid = GridSearchCV(randomf, params, n_jobs=-1, cv= 5, return_train_score=True,)
grid.fit(X_tr, y_tr)


In [ ]:
grid.best_estimator_.feature_importances_

max_depth=np.int64(21), n_estimators=1000   0.5669713893907954

max_depth=np.int64(19), min_samples_leaf=2, n_estimators=1000  0.5347813665813888

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_

In [ ]:
best_model = grid.best_estimator_

# 2. 변수 중요도를 계산합니다.
importances = best_model.feature_importances_

# 3. 컬럼 이름은 학습 데이터(X_tr)에서 바로 가져옵니다.
feature_names = X_tr.columns

# 4. 컬럼 이름과 중요도를 합쳐서 보기 좋게 출력합니다.
feature_importances = pd.Series(importances, index=feature_names).sort_values(ascending=False)

print("--- 변수 중요도 (상위 20개) ---")
print(feature_importances)

In [ ]:
score_history = []

results = pd.DataFrame(grid.cv_results_)
results['train-test'] = results['mean_train_score'] - results['mean_test_score']
results_10 = results[['param_max_depth', 'param_min_samples_leaf', 'param_n_estimators',
       'mean_test_score', 'std_test_score','train-test']].sort_values(by=['mean_test_score','train-test'], ascending=[False, True]).head(10)

score_history.extend([[results_10.iloc[0]]])

results_10

#XGBOOST

In [ ]:
from xgboost import XGBRegressor

In [ ]:
cv = cross_validate(xg,X_tr, y_tr, n_jobs= -1, return_train_score=True )
print(cv['train_score'].mean())
print(cv['test_score'].mean())

In [ ]:
params = {
    'learning_rate': [0.01, 0.03],
    'max_depth': [5, 6, 7],

    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

grid = GridSearchCV(xg, params, n_jobs=-1, cv= 5, return_train_score=True)
grid.fit(X_tr, y_tr)

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
results = pd.DataFrame(grid.cv_results_)
results['train-test'] = results['mean_train_score'] - results['mean_test_score']
results_10 = results[['param_colsample_bytree', 'param_learning_rate', 'param_max_depth',
        'param_subsample','mean_train_score', 'mean_test_score','train-test']].sort_values(by=['mean_test_score','train-test'], ascending=[False, True]).head(10)

results_10

In [ ]:
grid.best_params_